In [72]:
# # today = (Sunny, Hot, Normal, False) -> output should be YES
from sklearn import tree
import csv
import numpy as np

data = [] #to store the data(without the column that contains the class) after reading
target = [] # contains the class column
featureName = [] # the name of the columns, to use in different situations
targetName = [] # the unique values of classlabels
className = [] # name of the class


In [98]:
def readData( ):
    
    FILENAME = "rain.csv"
    COLUMNSTOIGNORE = [0]  # ignoring the useless columns, these columns will not be read
    NULLVALUEINDICATOR = [ -100000, "?"]
    
    global data, target, featureName, targetName, className
    tx = []
    ty = []
    labels = []
    columnName = []
    with open(FILENAME,encoding="utf8") as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for i, rows in enumerate(csv_reader):
#             if(i>5000):
#                 break #breaking early just for quick runs
            tempData = []
            for j,data in enumerate(rows): #reading every column info from a row and ignoring the unnecessary columns
                if j not in COLUMNSTOIGNORE:
                    tempData.append(data)
            nullvalueFound = 0
            for j in NULLVALUEINDICATOR:
                if(j in tempData):
                    nullvalueFound = 1
                    break
            if(nullvalueFound):
                continue
            if i>0: #0 th row should be the name row
                datas = tempData[:-1]  #last column is the label column so seperating those
                label = (tempData[-1:]) #taking just he labels
                #print(datas, label)
                tx.append(datas)
                ty.append(label)
            else:
                columnName.extend(tempData) #copying array
    data = tx
    target = ty
    mytemp = np.array(target)
    #print(np.unique(target).tolist())
    targetName = np.unique(target).tolist()
    featureName = columnName[:-1]
    className = columnName[-1:]
    print(className)

In [99]:
readData()

# print("Data In Order: ")
# print("Target names: ")
# print(targetName)
# print("Features")
# print(featureName)
# print("Data")
# print(data)
# print("Targets")
# print(target)

['Play Golf']


In [75]:
# changing categorical data to one hot
'''
Training the encoder here on all the data read from the file because 
if we train the encoder from only the traning dataset then when testing,
the encoder thus the model might face unknown valus that don't have any 
one hot value.
Like in testingdata there might be a column named Ghana but there is no
value named Ghana in training dataset. so the encoder/model won't know any value for Ghana
'''
from sklearn import preprocessing

dataEncoder = preprocessing.OneHotEncoder()
dataEncoder.fit(data)  #training the encoder

targetEncoder = preprocessing.OneHotEncoder()
targetEncoder.fit(target)


#generating combined feature name from the one hot conversion

# onehotFeatureNames = []
# for i, val in enumerate(dataEncoder.categories_):
# #     print(featureName[i])
#     for j in val:
#         onehotFeatureNames.append("{}:{}".format(featureName[i],j))
# print(onehotFeatureNames)

OneHotEncoder(categorical_features=None, categories=None, drop=None,
              dtype=<class 'numpy.float64'>, handle_unknown='error',
              n_values=None, sparse=True)

In [76]:
# seperating training data and testing data
# for training we use the variables data, target

from sklearn.utils import shuffle
import math

SPLIT = 70 # means: out of 100, 70 will go to training and 30 will go to testing

data, target = shuffle(np.array(data), np.array(target))

splitPoint = math.ceil( len(data)*SPLIT / 100.0 )
testingData = data[splitPoint:]
testingTarget = target[splitPoint:]
data = data[:splitPoint]
target = target[:splitPoint]

print("Training Data Size: ", len(data),"Testing Data Size: ", len(testingData))
# print(testingData, testingTarget)

Training Data Size:  10 Testing Data Size:  4


In [77]:
#converting the data using the previously trained encoder
oneHotData = dataEncoder.transform(data) #converting the data

oneHotTarget = targetEncoder.transform(target)
# print(encData.toarray())
# print(enc.categories_)
# print(eTarget.categories_)

In [78]:

#decision tree classifier

DECISIONTREE = tree.DecisionTreeClassifier()
DECISIONTREE = DECISIONTREE.fit(oneHotData.toarray(), oneHotTarget.toarray())
# tree.plot_tree(clf) 
# print(encData.toarray())
# print(encTarget.toarray())


In [79]:
# for showing the result, unfortunatly sklearn's decisiontree doesn't work well with categorical data
import graphviz 
# dot_data = tree.export_graphviz(clf, out_file=None) 
# graph = graphviz.Source(dot_data) 
# graph.render("rainy") 


dot_data = tree.export_graphviz(DECISIONTREE, out_file=None, 
            feature_names=onehotFeatureNames,  
            class_names=targetName,  
            filled=True, rounded=True,  
            special_characters=True)  
graph = graphviz.Source(dot_data) 
graph.render("rainyData")
 
# graph 

'rainyData.pdf'

In [80]:

#today = (Sunny, Hot, Normal, False)
trialData = [ ['Sunny', 'Hot', 'Normal', 'FALSE'] ]
print("Input: ",trialData)
trialData = dataEncoder.transform( trialData )  # transforming the given data in to our own one hot
prediction = DECISIONTREE.predict(trialData.toarray())
predictionResult = targetEncoder.inverse_transform(prediction)
print("Prediction: ", predictionResult[0][0])

Input:  [['Sunny', 'Hot', 'Normal', 'FALSE']]
Prediction:  Yes


In [81]:
# ERROR CALCULATION

correctAnswer = 0
# print(testingData)
testData = dataEncoder.transform( testingData )
prediction = DECISIONTREE.predict(testData.toarray())
predictionResult = targetEncoder.inverse_transform(prediction)

# print("Testing Data")
# print(testingData)
# print(testingData, data)
# print(testingTarget, predictionResult)

for i,val in enumerate(predictionResult):
    if(testingTarget[i] == predictionResult[i]):
        correctAnswer +=1
successRate = correctAnswer / len(testingTarget)
print("Success rate: {}%".format(successRate*100))

Success rate: 75.0%
